In [18]:
import ee
import geemap
geemap.set_proxy(port = 56940)
ee.Initialize()
# Create a map centered at (lat, lon).
Map = geemap.Map(center=[40, -100], zoom=4)

collection = ee.ImageCollection('LANDSAT/LC08/C01/T1')

point = ee.Geometry.Point(-122.262, 37.8719)
start = ee.Date('2014-06-01')
finish = ee.Date('2014-10-01')

filteredCollection = ee.ImageCollection('LANDSAT/LC08/C01/T1') \
    .filterBounds(point) \
    .filterDate(start, finish) \
    .sort('CLOUD_COVER', True)

first = filteredCollection.first()
# Define visualization parameters in an object literal.
vizParams = {'bands': ['B5', 'B4', 'B3'],
             'min': 5000, 'max': 15000, 'gamma': 1.3}
Map.addLayer(first, vizParams, 'Landsat 8 image')

wi = first.expression('(1.75*g-r-1.08*nir)**2/(g+nir)**2',{
    'b':first.select('B2'),
  'g':first.select('B3'),
  'r':first.select('B4'),
  'nir':first.select('B5'),
  'swir1':first.select('B6'),
  'swir2':first.select('B7')
})
vizParams2 = {'min': 0, 'max': 1, 'gamma': 1.3}
Map.addLayer(wi, vizParams2, 'wi')
# Display the map.
Map


Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [23]:
palette = ['000000', '0000FF', '00FF00', 'FF0000']
wi_mask = wi.updateMask(wi.lt(0.02))
Map.addLayer(wi_mask, {'palette ':palette}, 'wi_mask')